# NLP getting started Kaggle competition solution

## Pasos a seguir



*   Exploración de los datos
*   Valores nulos de la columna keyword se reemplazan con cat "neutral"
*   Se elimina variable location dado que los datos contienen mucho ruido



In [1]:
import numpy as np
import pandas as pd
#!pip install unidecode
from unidecode import unidecode
import re
from nltk.corpus import stopwords
import nltk
#!nltk.download('stopwords')
#!nltk.download('wordnet')
#!pip install kaggle
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
#nltk.download('stopwords')
#nltk.download('wordnet')
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [2]:
#!mkdir ~/.kaggle
#!cp kaggle.json ~/.kaggle/kaggle.json
#!chmod 600 ~/.kaggle/kaggle.json
#!kaggle competitions download -c nlp-getting-started
#!unzip nlp-getting-started.zip

## EDA

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
df['keyword'].value_counts()

fatalities               45
deluge                   42
armageddon               42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [6]:
def preprocessing_data(df):
  df['keyword'] = df['keyword'].fillna('neutral')
  df.drop('location', axis = 1, inplace = True)

  return df


def noTilde(text):
    # Eliminar caracteres no alfabéticos y transliterar caracteres especiales a caracteres ASCII
    processed_text = unidecode(text.lower())
    # Eliminar los dígitos y caracteres especiales utilizando una expresión regular
    clean_text = re.sub(r'[^a-z0-9 ]', ' ', processed_text)

    # Inicializar el lematizador de palabras
    lemmatizer = WordNetLemmatizer()

    # Lematizar cada palabra en el texto limpio
    lemmatized_text = ' '.join([lemmatizer.lemmatize(word,pos='v') for word in clean_text.split()])

    # Eliminar espacios adicionales y retornar el texto lematizado y limpio
    lemmatized_text = ' '.join(lemmatized_text.split())

    return lemmatized_text

In [7]:
abt = df.copy()

In [8]:
preprocessing_data(abt)
abt['keyword'] = abt['keyword'].apply(noTilde)

### Modelo



*   Train/test split
*   Preprocessing
*  Model
* Pipeline



In [9]:
abt.head()

,id,keyword,text,target
0,1,neutral,Our Deeds are the Reason of this #earthquake M...,1
1,4,neutral,Forest fire near La Ronge Sask. Canada,1
2,5,neutral,All residents asked to 'shelter in place' are ...,1
3,6,neutral,"13,000 people receive #wildfires evacuation or...",1
4,7,neutral,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
ids = abt['id']
X = abt.drop(['target','id'],axis=1)
y = abt['target']

In [11]:
X.shape,y.shape

((7613, 2), (7613,))

In [12]:
# Create training and testing partitions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [13]:
X_train

,keyword,text
4226,hazardous,Never fear quarrels but seek hazardous adventu...
4092,hail,Hail! [pic] ÛÓ https://t.co/B7omJ7U3EI
4150,harm,I don't pray harm on members of ISIS.I pray th...
1311,burn,Don't be so modest. You certainly... *sniff* *...
7053,typhoon,Obama Declares Disaster for Typhoon-Devastated...
...,...,...
3276,epicentre,@elisagxrcia I think of that every time I go t...
7221,weapons,Incredulous at continued outcry of welfare bei...
1318,burn,@nagel_ashley @Vicken52 @BasedLaRock @goonc1ty...
723,bleed,@CoreyAshe Did that look broken or bleeding?


In [14]:
def preprocess_text(text):
    # Tokenise to alphabetic tokens
    tokeniser = RegexpTokenizer(r'[A-Za-z]+')
    tokens = tokeniser.tokenize(text)

    # Lowercase and lemmatise
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(t.lower()) for t in tokens]

    # Remove stopwords
    stopWords = stopwords.words('english') + stopwords.words('french')
    tokens = [t for t in tokens if t not in stopWords]
    return tokens

def preprocessing_text(text):
# Eliminar caracteres no alfabéticos y transliterar caracteres especiales a caracteres ASCII
  processed_text = unidecode(text.lower())
  # Tokenizar el texto en palabras utilizando expresión regular excluyendo los dígitos
  # Lowercase and lemmatise
  lemmatiser = WordNetLemmatizer()
  tokens = re.findall(r'[^\W\d]+', processed_text)
  stopWords = stopwords.words('english')
  tokens = [t for t in tokens if t not in stopWords]
  tokens = [lemmatiser.lemmatize(t.lower()) for t in tokens]

  return tokens

In [15]:
#Esto se usa cuando solo es una variable independiente
#X_train = X_train.squeeze()
#X_test = X_test.squeeze()

In [24]:
text_column = 'text'
categorical_columns = ['keyword']

# Crear el preprocesamiento con CountVectorizer y OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('text_vectorizer', CountVectorizer(analyzer = preprocessing_text, dtype = np.float64, decode_error='ignore'), text_column),
        ('onehot_encoder', OneHotEncoder(), categorical_columns)
    ])




pip1 = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestClassifier(random_state=1234))
])


pip2= Pipeline([
    ('preprocessor', preprocessor),
    ('gb', GradientBoostingClassifier(random_state=1234))])


pipelines = {
    'rf':pip1,
    'gb':pip2
}


grid = {
    'rf':{
    'rf__n_estimators': [100, 200],
    #'rf__max_depth': [None, 5, 10],
    'rf__min_samples_split': [2, 5, 10],
    #'rf__min_samples_leaf': [1, 2, 4],
    'rf__max_features': ['auto', 'sqrt']
},
    'gb':{
    'gb__n_estimators': [100,200],
    'gb__learning_rate': [0.1, 0.5],
    #'gb__max_depth': [3, 5, None],
    'gb__min_samples_split': [2, 5, 10],
    #'gb__min_samples_leaf': [1, 2, 4],
    'gb__max_features': ['auto', 'sqrt']
}
}


In [ ]:
# Create a blank dictionary to hold the models
fit_models = {}
# Loop through all the algos
for algo, pipeline in pipelines.items():
  print(f'Training the {algo} model.')
  # Create new Grid Search CV Cclass
  model = GridSearchCV(pipeline, grid[algo], n_jobs=1, cv=6)
  # Train the model
  print('dimensiones')
  print(X_train.shape,y_train.shape)
  model.fit(X_train, y_train)
  # Store results inside of the dictionary
  fit_models[algo] = model

In [26]:
# Evaluate the performance of the model
for algo, model in fit_models.items():
  yhat = model.predict(X_test)
  accuracy = accuracy_score(y_test, yhat)
  precision = precision_score(y_test, yhat)
  recall = recall_score(y_test, yhat)
  f1 = f1_score(y_test,yhat)
  print(f'Metrics for {algo}: accuracy- {accuracy}, recall- {recall}, precision- {precision}, f1- {f1} ')

Metrics for rf: accuracy- 0.8073555166374781, recall- 0.6536144578313253, precision- 0.8726541554959786, f1- 0.7474167623421354 
Metrics for gb: accuracy- 0.7889667250437828, recall- 0.6646586345381527, precision- 0.817283950617284, f1- 0.7331118493909192 


In [27]:
import pickle
with open('randomforest.pkl', 'wb') as f:
  pickle.dump(fit_models['rf'], f)
with open('randomforest.pkl', 'rb') as f:
  reloaded_model = pickle.load(f)

In [30]:
# Read in the Test CSV Dataset
test_df = pd.read_csv('test.csv')
# Deep copy
abt_test = test_df.copy()
preprocessing_data(abt_test)
abt_test['keyword'] = abt_test['keyword'].apply(noTilde)
yhat_test = fit_models['rf'].predict(abt_test)

In [32]:
submission = pd.DataFrame([test_df['id'], yhat_test]).T
submission.columns = ['id', 'target']
submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1


In [33]:
submission.to_csv('kaggle_submission.csv', index=False)
!kaggle competitions submit -c nlp-getting-started -m "initial rf model" -f "kaggle_submission.csv"

100% 22.2k/22.2k [00:00<00:00, 123kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets